# Merge the all car details of the cities 

In [1]:
import pandas as pd

# Define file paths for each city's dataset
file_paths = [
    (r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\bangalore_cars.xlsx', 'Bangalore'),
    (r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\chennai_cars.xlsx', 'Chennai'),
    (r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\delhi_cars.xlsx', 'Delhi'),
    (r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\jaipur_cars.xlsx', 'Jaipur'),
    (r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\kolkata_cars.xlsx', 'Kolkata'),
    (r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\hyderabad_cars.xlsx', 'Hyderabad')
]

# Read and concatenate the files with the respective city as a new column
dfs = []
for file_path, city in file_paths:
    df = pd.read_excel(file_path)
    df['City'] = city  # Add a new column with the city name
    dfs.append(df)

Merged_Car_data = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the concatenated DataFrame
print(Merged_Car_data.head())

# Save the concatenated DataFrame to a new CSV file
Merged_Car_data.to_csv(r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\Processed files\merged_cars_data.csv', index=False)

print("All datasets concatenated and saved")
print(Merged_Car_data.columns)

                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': 'Features', 'top': [{'value': 'Pow...   
3  {'heading': 'Features', 'top': [{'value': 'Pow...   
4  {'heading': 'Features', 'top': [{'value': '

# Flatten the merged car data

In [3]:
import pandas as pd
import ast  # For safely evaluating string literals into Python objects

file_path = r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\Processed files\merged_cars_data.csv'

# Read the entire CSV file at once to avoid multiple reads
df = pd.read_csv(file_path, usecols=['new_car_detail', 'new_car_overview', 'new_car_specs', 'City'])


# Function to parse the nested data
def parse_car_data(data, section=''):
    try:
        car_data = ast.literal_eval(data)

        if section == 'overview':
            return {item['key']: item['value'] for item in car_data.get('top', [])}

        if section == 'feature':
            top_features = [item['value'] for item in car_data.get('top', [])]
            feature_data = {}
            for category in car_data.get('data', []):
                category_heading = category.get('heading', '')
                feature_data[f'{category_heading}_subHeading'] = category.get('subHeading', '')
                feature_data[f'{category_heading}_list'] = [item['value'] for item in category.get('list', [])]
            feature_data['commonIcon'] = car_data.get('commonIcon', '')
            feature_data['top_features'] = top_features
            return feature_data

        if section == 'specs':
            top_specs = {item['key']: item['value'] for item in car_data.get('top', [])}
            specs_data = {}
            for category in car_data.get('data', []):
                category_heading = category.get('heading', '')
                for item in category.get('list', []):
                    specs_data[f'{category_heading}_{item["key"]}'] = item['value']
            specs_data['commonIcon'] = car_data.get('commonIcon', '')
            specs_data.update(top_specs)
            return specs_data

        # Default behavior for 'detail' or unknown sections
        return car_data

    except Exception as e:
        print(f"Error parsing {section} data: {e}")
        return {}


# Apply the parsing function to each column, specifying the type of data being processed
df_detail_parsed = df['new_car_detail'].apply(parse_car_data, section='detail')
final_df_detail = pd.json_normalize(df_detail_parsed)
print("Processed Data Preview for new_car_detail:")
print(final_df_detail.head())

df_overview_parsed = df['new_car_overview'].apply(parse_car_data, section='overview')
final_df_Overview = pd.json_normalize(df_overview_parsed)
print("Processed Data Preview for new_car_overview:")
print(final_df_Overview.head())

df_specs_parsed = df['new_car_specs'].apply(parse_car_data, section='specs')
final_df_specs = pd.json_normalize(df_specs_parsed)
print("Processed Data Preview for new_car_specs:")
print(final_df_specs.head())

# Combine all the dataframes into one final dataframe
final_df = pd.concat([final_df_detail, final_df_Overview, final_df_specs, df['City']], axis=1)


# Save the result to a new CSV file or Excel file
output_file = r'C:\Users\ramya\OneDrive\Documents\Python Scripts\Car Dheko\Processed files\processed_car_data.csv'
final_df.to_csv(output_file, index=False)

# Print the final combined DataFrame
print("Final Combined Data Preview:")
print(final_df.head())


Processed Data Preview for new_car_detail:
   it      ft         bt        km transmission  ownerNo      owner      oem  \
0   0  Petrol  Hatchback  1,20,000       Manual        3  3rd Owner   Maruti   
1   0  Petrol        SUV    32,706       Manual        2  2nd Owner     Ford   
2   0  Petrol  Hatchback    11,949       Manual        1  1st Owner     Tata   
3   0  Petrol      Sedan    17,794       Manual        1  1st Owner  Hyundai   
4   0  Diesel        SUV    60,000       Manual        1  1st Owner   Maruti   

                model  modelYear  centralVariantId               variantName  \
0      Maruti Celerio       2015              3979                       VXI   
1       Ford Ecosport       2018              6087  1.5 Petrol Titanium BSIV   
2          Tata Tiago       2018              2983           1.2 Revotron XZ   
3       Hyundai Xcent       2014              1867        1.2 Kappa S Option   
4  Maruti SX4 S Cross       2015              4277             DDiS 200 Zeta